In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


import pandas as pd
import tensorflow as tf
# import theano
import keras 

import numpy as np
import matplotlib.pyplot as plt
from time import time

Using TensorFlow backend.


In [2]:
# import tensorflow as tf
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18010277239172468050
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 1630470144
locality {
  bus_id: 1
}
incarnation: 15107070675119206953
physical_device_desc: "device: 0, name: GeForce GTX 750 Ti, pci bus id: 0000:02:00.0"
]


In [4]:
# import tensorflow as tf
# with tf.device('/gpu:0'):
#     a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
#     b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
#     c = tf.matmul(a, b)

# with tf.Session() as sess:
#     print (sess.run(c))

In [5]:
!nvidia-smi

Wed Oct 25 13:36:20 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.82                 Driver Version: 375.82                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 750 Ti  Off  | 0000:02:00.0     Off |                  N/A |
| N/A   46C    P0    N/A /  N/A |   1779MiB /  2002MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
def show(id, w=6):
    from IPython.display import Image
    return Image('./pics/class1/{}.jpg'.format(id), width=w*100)

In [7]:
df = pd.read_csv('../Artificial_Neural_Networks/Churn_Modelling.csv')
df.shape

(10000, 14)

In [8]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [9]:
df.iloc[:,3:-1].head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [10]:
df.iloc[:,3:13].head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [11]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [12]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

### data preprocessing

In [13]:
X = df.iloc[:,3:-1].values
print X.shape
print type(X)
X[:3]

(10000, 10)
<type 'numpy.ndarray'>


array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57]], dtype=object)

In [14]:
y = df.iloc[:,-1].values
print y.shape
print type(y)
y

(10000,)
<type 'numpy.ndarray'>


array([1, 0, 1, ..., 1, 1, 0])

### Encoding categorical data

In [15]:
X[:,1:3]

array([['France', 'Female'],
       ['Spain', 'Female'],
       ['France', 'Female'],
       ..., 
       ['France', 'Female'],
       ['Germany', 'Male'],
       ['France', 'Female']], dtype=object)

In [16]:
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
print X[:3]
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
X[:3]

[[619 0 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 2 'Female' 41 1 83807.86 1 0 1 112542.58]
 [502 0 'Female' 42 8 159660.8 3 1 0 113931.57]]


array([[619, 0, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 2, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 0, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57]], dtype=object)

In [17]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ..., 
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [18]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
print X.shape

(10000, 12)


In [19]:
onehotencoder.fit_transform(X)

<10000x13 sparse matrix of type '<type 'numpy.float64'>'
	with 91124 stored elements in COOrdinate format>

In [20]:
# %precision % .2f

https://stackoverflow.com/questions/2891790/how-to-pretty-printing-a-numpy-array-without-scientific-notation-and-with-given

In [21]:
np.set_printoptions(precision=1, suppress=True)

In [22]:
onehotencoder.fit_transform(X).toarray()[0]

array([      1. ,       0. ,       1. ,       0. ,     619. ,       0. ,
            42. ,       2. ,       0. ,       1. ,       1. ,       1. ,
        101348.9])

### remove one dummy variable to avoid dummy variable trap

In [23]:
X[0]

array([      1. ,       0. ,       0. ,     619. ,       0. ,      42. ,
             2. ,       0. ,       1. ,       1. ,       1. ,  101348.9])

In [24]:
print X.shape
X = X[:, 1:]
print X.shape
X[0]

(10000, 12)
(10000, 11)


array([      0. ,       0. ,     619. ,       0. ,      42. ,       2. ,
             0. ,       1. ,       1. ,       1. ,  101348.9])

### train test split & feature scaling 

In [25]:
S = lambda *x: [i.shape for i in x]

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print S(X_train,X_test, y_train, y_test)

[(8000, 11), (2000, 11), (8000,), (2000,)]


In [27]:
X_train[0]

array([      0. ,       1. ,     667. ,       0. ,      34. ,       5. ,
             0. ,       2. ,       1. ,       0. ,  163830.6])

In [28]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print S(X_train,X_test, y_train, y_test)
X_train[0]

[(8000, 11), (2000, 11), (8000,), (2000,)]


array([-0.6,  1.7,  0.2, -1.1, -0.5,  0. , -1.2,  0.8,  0.6, -1. ,  1.1])

## build ANN

In [29]:
# show(13, w=8)

In [30]:
from keras.models import Sequential
from keras.layers import Dense

In [31]:
classifier = Sequential()
# classifier?

In [32]:
# first hidden layer
# classifier.add(Dense(units = 6, input_shape=(11,),  kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units = 6, input_dim=11,  kernel_initializer='uniform', activation='relu'))

In [33]:
# second hidden layer
classifier.add(Dense(units = 6,  kernel_initializer='uniform', activation='relu'))

In [34]:
# ouput layer
classifier.add(Dense(units = 1,  kernel_initializer='uniform', activation='sigmoid'))

In [35]:
# show(19, w=8)

In [36]:
# compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [37]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [38]:
!ps aux --sort=-%mem | awk 'NR<=6{print $0}'

USER       PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root      5140  6.1  3.5 1211740 583540 ?      Sl   13:34   0:08 gunicorn: worker [app:app]                                                                  
soyoung   5234  100  2.5 32442488 409104 ?     Ssl  13:36   0:03 /home/saya/anaconda2/bin/python -m ipykernel_launcher -f /home/soyoung/.local/share/jupyter/runtime/kernel-6f2273e5-3fbe-4642-85da-fef6f960b3b0.json
guest-l+  4435  1.6  1.6 1589060 275420 ?      Sl   13:31   0:05 compiz
guest-l+  4127  4.8  0.6 231556 108964 ?       Sl   13:31   0:14 /opt/teamviewer/tv_bin/TeamViewer_Desktop --IPCport 5939 --module 1
root      3166  0.4  0.6 302444 105028 ?       SNl  13:25   0:03 /usr/bin/python3 /usr/sbin/aptd


In [39]:
# tf.all_variables()

In [ ]:
# if 'session' in locals() and session is not None:
#     print('Close interactive session')
#     session.close()

In [ ]:
# fitting ANN with training set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 33s - loss: 0.4898 - acc: 0.7954    
Epoch 2/100
8000/8000 [==============================] - 2s - loss: 0.4275 - acc: 0.7960     
Epoch 3/100
8000/8000 [==============================] - 2s - loss: 0.4223 - acc: 0.7960     
Epoch 4/100
8000/8000 [==============================] - 2s - loss: 0.4184 - acc: 0.8208     
Epoch 5/100
8000/8000 [==============================] - 2s - loss: 0.4158 - acc: 0.8275     
Epoch 6/100
8000/8000 [==============================] - 2s - loss: 0.4146 - acc: 0.8306     
Epoch 7/100
8000/8000 [==============================] - 2s - loss: 0.4128 - acc: 0.8299     
Epoch 8/100
8000/8000 [==============================] - 2s - loss: 0.4120 - acc: 0.8318     
Epoch 9/100
8000/8000 [==============================] - 2s - loss: 0.4109 - acc: 0.8338     
Epoch 10/100
6140/8000 [======================>.......] - ETA: 0s - loss: 0.4093 - acc: 0.8308

In [ ]:
y_pred = classifier.predict(X_test)
print type(y_pred)
y_pred.shape

In [ ]:
y_pred

In [ ]:
y_pred[0][0]

In [ ]:
y_pred = (y_pred> 0.5)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
type(cm)

In [ ]:
cm

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
(1512+211)/2000.0

In [ ]:
classifier.save?

### Predicting a single new observation

In [ ]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

## cross validation